In [1]:
# Instalando as libs necessárias.
!pip install fuzzywuzzy 

In [2]:
# Importando os módulos.
import pandas as pd
import numpy as np
import pandas_profiling
import nltk
import warnings
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
warnings.filterwarnings("ignore")
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Dataset principal.
df = pd.read_csv("drive/My Drive/Colab Notebooks/NLP/wiki_movie_plots_deduped.csv")

In [0]:
# Importando as colunas a serem utilizadas.
df = pd.DataFrame(df,columns=["Title","Genre","Plot"])

In [0]:
# Vamos utilizar um outro Dataset com os genêros já padronizados.
df_2 = pd.read_csv("drive/My Drive/Colab Notebooks/NLP/movie_metadata.csv")

In [0]:
# Queremos somente a coluna com a descrição dos genêros.
df_2 = pd.DataFrame(df_2,columns=["genres"])

Visualizando o primeiro DataFrame.

In [8]:
df.head(20)

,Title,Genre,Plot
0,Kansas Saloon Smashers,unknown,"A bartender is working at a saloon, serving dr..."
1,Love by the Light of the Moon,unknown,"The moon, painted with a smiling face hangs ov..."
2,The Martyred Presidents,unknown,"The film, just over a minute long, is composed..."
3,"Terrible Teddy, the Grizzly King",unknown,Lasting just 61 seconds and consisting of two ...
4,Jack and the Beanstalk,unknown,The earliest known adaptation of the classic f...
5,Alice in Wonderland,unknown,"Alice follows a large white rabbit down a ""Rab..."
6,The Great Train Robbery,western,The film opens with two bandits breaking into ...
7,The Suburbanite,comedy,The film is about a family who move to the sub...
8,The Little Train Robbery,unknown,The opening scene shows the interior of the ro...
9,The Night Before Christmas,unknown,Scenes are introduced using lines of the poem....


Visualizando o segundo DataFrame.

In [9]:
df_2.head(20)

,genres
0,Action|Adventure|Fantasy|Sci-Fi
1,Action|Adventure|Fantasy
2,Action|Adventure|Thriller
3,Action|Thriller
4,Documentary
5,Action|Adventure|Sci-Fi
6,Action|Adventure|Romance
7,Adventure|Animation|Comedy|Family|Fantasy|Musi...
8,Action|Adventure|Sci-Fi
9,Adventure|Family|Fantasy|Mystery


* A coluna "Genre" do primeiro DataFrame é onde se encontra os dados com problemas de escrita.
* Na coluna "genres" os dados já estão limpos e devidamente padronizados.

In [0]:
df_ = df_2.drop_duplicates()

Vamos dar uma olhada na quantidade de valores únicos que as essas colunas possuem.

In [11]:
 print(df["Genre"].unique().shape[0])

2265


In [12]:
 print(df_2["genres"].unique().shape[0])

914


Visualizando alguns dados.

In [13]:
df["Genre"].unique().tolist()[:20]

['unknown',
 'western',
 'comedy',
 'short',
 'short action/crime western',
 'short film',
 'biographical',
 'drama',
 'adventure',
 'short fantasy',
 'silent sports',
 'horror',
 'crime',
 'drama, horror',
 'historical drama',
 'fantasy drama',
 'biographical drama',
 'documentary drama',
 'fantasy',
 'adventure serial']

In [14]:
df_2["genres"].unique().tolist()[:20]

['Action|Adventure|Fantasy|Sci-Fi',
 'Action|Adventure|Fantasy',
 'Action|Adventure|Thriller',
 'Action|Thriller',
 'Documentary',
 'Action|Adventure|Sci-Fi',
 'Action|Adventure|Romance',
 'Adventure|Animation|Comedy|Family|Fantasy|Musical|Romance',
 'Adventure|Family|Fantasy|Mystery',
 'Action|Adventure',
 'Action|Adventure|Western',
 'Action|Adventure|Family|Fantasy',
 'Action|Adventure|Comedy|Family|Fantasy|Sci-Fi',
 'Adventure|Fantasy',
 'Action|Adventure|Drama|History',
 'Adventure|Family|Fantasy',
 'Action|Adventure|Drama|Romance',
 'Drama|Romance',
 'Action|Adventure|Sci-Fi|Thriller',
 'Action|Adventure|Fantasy|Romance']

Nossa coluna "Genre" possui dados que estão separados por vírgula, alguns por traços, ou seja, não existe nenhum padrão. Podemos resolver isso facilmente removendo todos os caracteres que são estranhos.

In [0]:
df["Genre"] = df["Genre"].str.replace('[^A-Za-z]+', ' ')

Temos também valores que são indefinidos, nesse caso vamos transformar esse valores em branco.

In [0]:
df["Genre"] = df["Genre"].str.replace('unknown', ' ')

Vamos verificar como ficou nosso dataset depois dessas transformações.

In [17]:
df["Genre"].unique().tolist()[:20]

[' ',
 'western',
 'comedy',
 'short',
 'short action crime western',
 'short film',
 'biographical',
 'drama',
 'adventure',
 'short fantasy',
 'silent sports',
 'horror',
 'crime',
 'drama horror',
 'historical drama',
 'fantasy drama',
 'biographical drama',
 'documentary drama',
 'fantasy',
 'adventure serial']

Agora vamos criar uma função para aplicar a biblioteca fuzzywuzzy.
Esta função utiliza a biblioteca fuzzywuzzy para comparar cada genêro com
uma String que está correta, caso haja algum problema de grafia a string é substituída pela forma correta.

In [0]:
def replace_matches_in_column(data, column, string_match, min_ratio, only_test=False):

    # lista de strings únicas.
    strings = data[column].unique()

    # Definindo 10 como o número limite de strings mais correspondentes.
    matches = process.extract(string_match, strings,
                                         limit=10,
                                         scorer=fuzz.token_sort_ratio)

    # Obtendo as correspondencias que são maiores que o min_ratio.
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    if len(close_matches) > 0:
        print(string_match, ':', close_matches)

    # Obtendo as linhas que não correspondem ao limite definido.
    if not(only_test):
        rows_with_matches = data[column].isin(close_matches)

        # Substituindo todos os valores correspondentes pelo valor de entrada.
        # Os valores que não são correspondentes irão permanecer os mesmos.
        data.loc[rows_with_matches, column] = string_match

In [19]:
# Eu utilizei uma taxa de semelhança  de 80%.
for marca in df_2['genres']:
     replace_matches_in_column(data=df, column="Genre", 
                             string_match=marca, min_ratio=80)

Action|Adventure|Fantasy|Sci-Fi : ['action adventure fantasy sci fi', 'action adventure science fantasy', 'action adventure fantasy', 'fantasy action adventure', 'sci fi adventure drama action', 'superhero action adventure fantasy sci fi', 'action adventure sci fi', 'sci fi action adventure', 'action sci fi adventure', 'action adventure fantasy horror']
Action|Adventure|Fantasy : ['action adventure drama fantasy', 'drama action fantasy adventure', 'Action|Adventure|Fantasy|Sci-Fi', 'action adventure fantasy family', 'action adventure fantasy comedy', 'action fantasy adventure comedy', 'action adventure family', 'fantasy adventure', 'adventure fantasy', 'action adventure drama']
Action|Adventure|Thriller : ['action adventure thriller', 'action adventure thriller spy', 'action thriller adventure drama', 'action crime adventure', 'action adventure crime', 'adventure thriller', 'thriller adventure', 'action adventure horror', 'action adventure western', 'anime action adventure']
Action|Thr

Agora que terminamos de aplicar a nossa função e encontrar a similaridade entre os nossos dados, vamos verificar quantos valores únicos nosso dataset possui agora.

In [20]:
print(df["Genre"].unique().shape[0])

994


In [21]:
df.head(20)

,Title,Genre,Plot
0,Kansas Saloon Smashers,,"A bartender is working at a saloon, serving dr..."
1,Love by the Light of the Moon,,"The moon, painted with a smiling face hangs ov..."
2,The Martyred Presidents,,"The film, just over a minute long, is composed..."
3,"Terrible Teddy, the Grizzly King",,Lasting just 61 seconds and consisting of two ...
4,Jack and the Beanstalk,,The earliest known adaptation of the classic f...
5,Alice in Wonderland,,"Alice follows a large white rabbit down a ""Rab..."
6,The Great Train Robbery,Western,The film opens with two bandits breaking into ...
7,The Suburbanite,Comedy,The film is about a family who move to the sub...
8,The Little Train Robbery,,The opening scene shows the interior of the ro...
9,The Night Before Christmas,,Scenes are introduced using lines of the poem....


Bem, podemos observar agora que tivemos uma redução consideravel no número de valores únicos em nosso dataset.

Na verdade muitos desses valores significavam a mesma coisa, porém estavam escritos de várias formas diferentes. Ainda possuimos alguns valores incorretos, mais a sua grande maioria foi tratada.